In [ ]:
from uuid import UUID

from flask import Flask, render_template, redirect, url_for, jsonify, request

from game_lib import BASIC_ITEMS, Character

In [ ]:
all_items = BASIC_ITEMS

In [ ]:
bob = Character("Bob", 100)
bob.inventory.items = [all_items[0], all_items[1]]

carl = Character("Carl", 100)

lydia = Character("Lydia", 100)
lydia.inventory.items = [all_items[0], all_items[1], all_items[2]]

all_characters = [bob, carl, lydia]

In [ ]:
app = Flask(__name__, template_folder='templates')

In [ ]:
@app.route('/')
def index():
    return redirect(url_for('home'))


@app.route('/home')
def home():
    return render_template('home.html')


@app.route('/items')
def items():
    # TODO create new items
    # TODO remove existing items
    return render_template('items.html', items=all_items)


@app.route('/items/<item_id>')
def item_details(item_id):
    # TODO edit item
    item_uuid = UUID(item_id)
    item = next(filter(lambda x: x.id == item_uuid, all_items), None)
    return render_template('item_details.html', item=item)


@app.route('/characters')
def characters():
    # TODO create new characters
    # TODO remove existing characters
    return render_template('characters.html', characters=all_characters)


@app.route('/characters/<character_id>')
def character_details(character_id):
    # TODO edit character's name and health
    # TODO explore and change the character's inventory
    character_uuid = UUID(character_id)
    character = next(filter(lambda x: x.id == character_uuid, all_characters), None)
    return render_template('character_details.html', character=character)


@app.route('/characters/<character_id>/gold', methods=['POST'])
def character_modify_gold(character_id):
    character_uuid = UUID(character_id)
    character = next(filter(lambda x: x.id == character_uuid, all_characters), None)

    data = request.get_json()

    action = data.get('action')
    amount = data.get('amount')

    try:
        amount = int(amount)
    except ValueError:
        return jsonify({'message': 'Amount must be a number.'}), 400

    if amount < 0:
        return jsonify({'message': 'Amount must be positive.'}), 400

    if action == 'add':
        character.inventory.gold.amount += amount
        message = 'Gold was added.'
    elif action == 'remove':
        if amount > character.inventory.gold.amount:
            return jsonify({'message': 'Not enough gold.'}), 400
        character.inventory.gold.amount -= amount
        message = 'Gold was removed.'
    else:
        return jsonify({'message': f'Invalid action (action={action}).'}), 400

    return jsonify({
        'message': message,
        'updated_gold': character.inventory.gold.amount
    }), 200

In [ ]:
app.run()